In [1]:
import keras
import keras.backend as K
import kapre
import pandas as pd
import scipy.io.wavfile as wav
import numpy as np
import random
import arrow
import threading
from soph import soph_scaler, center_wave, ex_generator

# https://github.com/keunwoochoi/kapre

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [2]:
ex_df = pd.read_pickle("data/ex_df.pkl")
current_time = arrow.now().to('US/Eastern').format('YYYY-MM-DD-HH-mm')


src_args = {
    "dropout_prob": .4,
    "activation": "elu",
    "batch_size": 64,
    "l2_reg": 0.0005,
    "cnn_blocks": [
        [[64, 3]],
        [[64, 3]],
        [[128, 3]],
        [[128, 3]],
    ],
    "mel_trainable": False,
    "train_state": ["train", "val"],
    "val_state": ["test"],
    "p_transform": 0,
    "vol_range": 0,
    "displacement": 0,
    "shift":0,
    "n_mels": 26,
    "n_dft": 1024,
}
num_cat = 12
print(src_args)
    

{'displacement': 0, 'l2_reg': 0.0005, 'val_state': ['test'], 'n_mels': 26, 'cnn_blocks': [[[64, 3]], [[64, 3]], [[128, 3]], [[128, 3]]], 'p_transform': 0, 'batch_size': 64, 'shift': 0, 'n_dft': 1024, 'dropout_prob': 0.4, 'vol_range': 0, 'train_state': ['train', 'val'], 'activation': 'elu', 'mel_trainable': False}


In [3]:
act = src_args["activation"]
kern = (3, 3)
drop = src_args["dropout_prob"]
cnn_blocks = src_args["cnn_blocks"]

input_layers = [
    keras.layers.InputLayer(input_shape=(1, 16000)),
    kapre.time_frequency.Melspectrogram(
        sr=16000,
        n_mels=src_args["n_mels"],
        return_decibel_melgram=True,
        n_dft=src_args["n_dft"]),
]

cnn_layers = []

for block in cnn_blocks:
    for layer in block:
        cnn_layers.extend([
            keras.layers.Conv2D(
                layer[0], layer[1], padding="same", activation=act),
        ])
    cnn_layers.extend([
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(drop),
        keras.layers.AvgPool2D(padding='valid'),
    ])

class_layers = [
    keras.layers.Flatten(),
    keras.layers.Dense(num_cat, activation="softmax"),
]
cnn_model = keras.Sequential(input_layers + cnn_layers + class_layers)
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 16000)          0         
_________________________________________________________________
melspectrogram_1 (Melspectro (None, 26, 32, 1)         1063962   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 32, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 26, 32, 64)        0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 13, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 16, 64)        36928     
__________

In [28]:
log_base = "logs/cnn/{}/".format(current_time)
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir= log_base+'tb',
        batch_size=src_args["batch_size"],
        histogram_freq=0,
        write_grads=False, 
        write_images=True
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=log_base+'model-checkpoint.hdf5',
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        mode='auto',
        period=1),
    keras.callbacks.CSVLogger(log_base+'training.log'),
    keras.callbacks.EarlyStopping(patience=5, verbose=1),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.5, patience=1, verbose=1, min_lr=1e-6)
]

cnn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy'])

def launchTensorBoard():
    import os
    os.system('pkill tensorboard')
    os.system('tensorboard --logdir=' + log_base+'tb')
    return

t = threading.Thread(target=launchTensorBoard, args=([]))
t.start()

val_data = next(ex_generator(
        batch_size=sum(ex_df.state.isin(src_args["val_state"])),
        shuffle=False,
        state=src_args["val_state"],
        vol_range=0,
        displacement=0,
        p_transform=0))

cnn_model.fit_generator(
    generator=ex_generator(
        batch_size=src_args["batch_size"],
        shuffle=True,
        state=src_args["train_state"],
        shift=src_args["shift"],
        vol_range=src_args["vol_range"],
        displacement=src_args["displacement"],
        p_transform=src_args["p_transform"]),
    steps_per_epoch=sum(ex_df.state.isin(src_args["train_state"])) / src_args["batch_size"],
    epochs=200,
    verbose=1,
    max_queue_size=100,
    callbacks=callbacks,
    validation_data=val_data
)

Epoch 1/200
937/936 [==============================] - 97s 104ms/step - loss: 1.0777 - acc: 0.4818 - val_loss: 0.7547 - val_acc: 0.6606
Epoch 2/200
937/936 [==============================] - 39s 42ms/step - loss: 0.4802 - acc: 0.7221 - val_loss: 0.4818 - val_acc: 0.7070
Epoch 3/200
937/936 [==============================] - 39s 41ms/step - loss: 0.3902 - acc: 0.7744 - val_loss: 0.8317 - val_acc: 0.8093
Epoch 4/200
935/936 [============================>.] - ETA: 0s - loss: 0.3500 - acc: 0.8021Epoch 00004: val_loss did not improve

Epoch 00004: reducing learning rate to 0.0010000000474974513.
937/936 [==============================] - 39s 42ms/step - loss: 0.3497 - acc: 0.8019 - val_loss: 0.6128 - val_acc: 0.8117
Epoch 5/200
937/936 [==============================] - 39s 41ms/step - loss: 0.2836 - acc: 0.8341 - val_loss: 0.2145 - val_acc: 0.8717
Epoch 6/200
937/936 [==============================] - 39s 41ms/step - loss: 0.2692 - acc: 0.8485 - val_loss: 0.2817 - val_acc: 0.8403
Epoch 7/2

Epoch 28/200
937/936 [==============================] - 39s 41ms/step - loss: 0.1528 - acc: 0.9047 - val_loss: 0.1428 - val_acc: 0.9240
Epoch 29/200
935/936 [============================>.] - ETA: 0s - loss: 0.1512 - acc: 0.9057Epoch 00029: val_loss did not improve

Epoch 00029: reducing learning rate to 3.125000148429535e-05.
937/936 [==============================] - 38s 41ms/step - loss: 0.1510 - acc: 0.9056 - val_loss: 0.1422 - val_acc: 0.9214
Epoch 30/200
935/936 [============================>.] - ETA: 0s - loss: 0.1527 - acc: 0.9075Epoch 00030: val_loss did not improve

Epoch 00030: reducing learning rate to 1.5625000742147677e-05.
937/936 [==============================] - 39s 41ms/step - loss: 0.1527 - acc: 0.9076 - val_loss: 0.1414 - val_acc: 0.9242
Epoch 31/200
937/936 [==============================] - 39s 41ms/step - loss: 0.1481 - acc: 0.9069 - val_loss: 0.1396 - val_acc: 0.9228
Epoch 32/200
937/936 [==============================] - 39s 41ms/step - loss: 0.1507 - acc: 0.9